In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from datetime import datetime
from datetime import timedelta
%load_ext line_profiler
sns.set(style="darkgrid")
import requests
import pprint
import re
%matplotlib inline

In [4]:
time1 = datetime.now()
filename = '../cleaned_flight_data.csv'
p = 0.01  # p% of lines
random.seed(234)
df = pd.read_csv(
         filename,
         header=0, parse_dates = ['FL_DATE','dep_datetime','arr_datetime'],
         skiprows=lambda i: i>0 and random.random() > p
)
time2 = datetime.now()
print('Time cost: ', time2-time1)

Time cost:  0:00:24.427292


C:\Users\2pawn\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (77,84,104,105,106,108,110,114,115,116,118,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
time1 = datetime.now()
df['HOURLYVISIBILITY_origin'] = df['HOURLYVISIBILITY_origin'].str.replace('[^\d(.)]', "")
df['HOURLYVISIBILITY_dest'] = df['HOURLYVISIBILITY_dest'].str.replace('[^\d(.)]', "")
time2 = datetime.now()
print('Time cost: ', time2-time1)

Time cost:  0:00:00.150963


In [6]:
pd.set_option('display.max_columns', 500)
df.head(15)

YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK    FL_DATE  \
0   2016        1      1            29            5 2016-01-29   
1   2016        1      1            29            5 2016-01-29   
2   2016        2      4            26            2 2016-04-26   
3   2016        2      6            18            6 2016-06-18   
4   2016        2      6            18            6 2016-06-18   
5   2016        2      6            18            6 2016-06-18   
6   2016        3      7             3            7 2016-07-03   
7   2016        3      7             3            7 2016-07-03   
8   2016        3      7             3            7 2016-07-03   
9   2016        3      7             3            7 2016-07-03   
10  2016        3      7             6            3 2016-07-06   
11  2016        3      7             6            3 2016-07-06   
12  2016        3      7             6            3 2016-07-06   
13  2016        3      7             6            3 2016-07-06   
14  2016        3      7             6            3 2016-07-06   

   OP_UNIQUE_CARRIER  OP_CARRIER_AIRLINE_ID OP_CARRIER TAIL_NUM  \
0                 DL                  19790         DL   N635DL   
1                 WN                  19393         WN   N760SW   
2                 OO                  20304         OO   N745SK   
3                 WN                  19393         WN   N8627B   
4                 DL                  19790         DL   N939DN   
5                 OO                  20304         OO   N978SW   
6                 AS                  19930         AS   N607AS   
7                 AS                  19930         AS   N570AS   
8                 WN                  19393         WN   N792SW   
9                 UA                  19977         UA   N62883   
10                F9                  20436         F9   N941FR   
11                OO                  20304         OO   N121SY   
12                DL                  19790         DL   N320US   
13                OO                  20304         OO   N8884E   
14                AA                  19805         AA   N3GFAA   

    OP_CARRIER_FL_NUM  ORIGIN_AIRPORT_ID  ORIGIN_AIRPORT_SEQ_ID  \
0                 903              10397                1039705   
1                2885              12191                1219102   
2                5304              11298                1129804   
3                5043              14057                1405702   
4                1959              11433                1143302   
5                5543              10713                1071302   
6                 384              14057                1405702   
7                   6              12892                1289204   
8                1514              13342                1334205   
9                 420              12892                1289204   
10                660              14771                1477102   
11               5364              11109                1110902   
12                932              14869                1486903   
13               4806              11986                1198603   
14               1084              13303                1330303   

    ORIGIN_CITY_MARKET_ID ORIGIN       ORIGIN_CITY_NAME ORIGIN_STATE_ABR  \
0                   30397    ATL            Atlanta, GA               GA   
1                   31453    HOU            Houston, TX               TX   
2                   30194    DFW  Dallas/Fort Worth, TX               TX   
3                   34057    PDX           Portland, OR               OR   
4                   31295    DTW            Detroit, MI               MI   
5                   30713    BOI              Boise, ID               ID   
6                   34057    PDX           Portland, OR               OR   
7                   32575    LAX        Los Angeles, CA               CA   
8                   33342    MKE          Milwaukee, WI               WI   
9                   32575    LAX        L

In [8]:
import math
def spacetonan(x):
    if isinstance(x, str) and x != '':
        return float(x)
    elif x == '':
        return np.nan
    return x

In [9]:
df['HOURLYVISIBILITY_origin'] = list(map(spacetonan, df['HOURLYVISIBILITY_origin']))

In [10]:
df['HOURLYVISIBILITY_origin'].unique()

array([10.  ,   nan,  8.  ,  1.  ,  9.  ,  6.  ,  7.  ,  5.  ,  3.  ,
        4.  ,  2.  ,  0.5 ,  2.5 ,  0.25,  1.5 ,  0.12,  0.75,  1.25,
        1.75,  0.  , 20.  , 15.  ,  6.21])

In [13]:
np.set_printoptions(threshold=np.nan)
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit([df['HOURLYVISIBILITY_origin']])
X = [df['HOURLYVISIBILITY_origin']]
X = imp_mean.transform(X)

In [20]:
np.unique(X)

array([ 0.  ,  0.12,  0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,
        2.  ,  2.5 ,  3.  ,  4.  ,  5.  ,  6.  ,  6.21,  7.  ,  8.  ,
        9.  , 10.  , 15.  , 20.  ])

In [33]:
np.set_printoptions(threshold=np.nan)
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df['HOURLYVISIBILITY_origin_imp'] = imp_mean.fit_transform(df[['HOURLYVISIBILITY_origin']]).ravel()

In [35]:
df['HOURLYVISIBILITY_origin_imp'].unique()

array([10.        ,  9.44614598,  8.        ,  1.        ,  9.        ,
        6.        ,  7.        ,  5.        ,  3.        ,  4.        ,
        2.        ,  0.5       ,  2.5       ,  0.25      ,  1.5       ,
        0.12      ,  0.75      ,  1.25      ,  1.75      ,  0.        ,
       20.        , 15.        ,  6.21      ])

In [34]:
df.head(10)

YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK    FL_DATE  \
0  2016        1      1            29            5 2016-01-29   
1  2016        1      1            29            5 2016-01-29   
2  2016        2      4            26            2 2016-04-26   
3  2016        2      6            18            6 2016-06-18   
4  2016        2      6            18            6 2016-06-18   
5  2016        2      6            18            6 2016-06-18   
6  2016        3      7             3            7 2016-07-03   
7  2016        3      7             3            7 2016-07-03   
8  2016        3      7             3            7 2016-07-03   
9  2016        3      7             3            7 2016-07-03   

  OP_UNIQUE_CARRIER  OP_CARRIER_AIRLINE_ID OP_CARRIER TAIL_NUM  \
0                DL                  19790         DL   N635DL   
1                WN                  19393         WN   N760SW   
2                OO                  20304         OO   N745SK   
3                WN                  19393         WN   N8627B   
4                DL                  19790         DL   N939DN   
5                OO                  20304         OO   N978SW   
6                AS                  19930         AS   N607AS   
7                AS                  19930         AS   N570AS   
8                WN                  19393         WN   N792SW   
9                UA                  19977         UA   N62883   

   OP_CARRIER_FL_NUM  ORIGIN_AIRPORT_ID  ORIGIN_AIRPORT_SEQ_ID  \
0                903              10397                1039705   
1               2885              12191                1219102   
2               5304              11298                1129804   
3               5043              14057                1405702   
4               1959              11433                1143302   
5               5543              10713                1071302   
6                384              14057                1405702   
7                  6              12892                1289204   
8               1514              13342                1334205   
9                420              12892                1289204   

   ORIGIN_CITY_MARKET_ID ORIGIN       ORIGIN_CITY_NAME ORIGIN_STATE_ABR  \
0                  30397    ATL            Atlanta, GA               GA   
1                  31453    HOU            Houston, TX               TX   
2                  30194    DFW  Dallas/Fort Worth, TX               TX   
3                  34057    PDX           Portland, OR               OR   
4                  31295    DTW            Detroit, MI               MI   
5                  30713    BOI              Boise, ID               ID   
6                  34057    PDX           Portland, OR               OR   
7                  32575    LAX        Los Angeles, CA               CA   
8                  33342    MKE          Milwaukee, WI               WI   
9                  32575    LAX        Los Angeles, CA               CA   

   ORIGIN_STATE_FIPS ORIGIN_STATE_NM  ORIGIN_WAC  DEST_AIRPORT_ID  \
0                 13         Georgia          34            10721   
1                 48           Texas          74            11697   
2                 48           Texas          74            13930   
3                 41          Oregon          92            11292   
4                 26        Michigan          43            15304   
5                 16           Idaho          83            14771   
6                 41          Oregon          92            14771   
7                  6      California          91            11278   
8                 55       Wisconsin          45            11278   
9                  6      California          91            13930   

   DEST_AIRPORT_SEQ_ID  DEST_CITY_MARKET_ID DEST       DEST_CITY_NAME  \
0              1072102                30721  BOS           Boston, MA   
1              1169704                32467  FLL  Fort Lauderdale, FL   
2              1393004      